In [1]:
import comp_utils

rebase_api_client = comp_utils.RebaseAPI(api_key = open("team_key.txt").read())

In [3]:
import pandas as pd
market_day = pd.DataFrame({"datetime":comp_utils.day_ahead_market_times(today_date=pd.to_datetime("2024-10-05"))})
# market_day = market_day.merge(submission_date_forecast,how="left",left_on="datetime",right_on="valid_datetime")
# market_day["market_bid"] = market_day["q50"]
market_day

,datetime
0,2024-10-05 23:00:00+01:00
1,2024-10-05 23:30:00+01:00
2,2024-10-06 00:00:00+01:00
3,2024-10-06 00:30:00+01:00
4,2024-10-06 01:00:00+01:00
5,2024-10-06 01:30:00+01:00
6,2024-10-06 02:00:00+01:00
7,2024-10-06 02:30:00+01:00
8,2024-10-06 03:00:00+01:00
9,2024-10-06 03:30:00+01:00


In [2]:
test_date = "2024-10-02"

rebase_api_client.get_variable(day=test_date,variable="market_index").head()

,timestamp_utc,settlement_date,settlement_period,data_provider,price,volume
0,2024-10-01T22:00:00Z,2024-10-01,47,APXMIDP,80.90,1027.40
1,2024-10-01T22:30:00Z,2024-10-01,48,APXMIDP,74.20,1258.25
2,2024-10-01T23:00:00Z,2024-10-02,1,APXMIDP,77.43,1243.20
3,2024-10-01T23:30:00Z,2024-10-02,2,APXMIDP,77.86,1268.75
4,2024-10-02T00:00:00Z,2024-10-02,3,APXMIDP,67.02,1297.75


In [5]:
import requests

your_api_key = open("team_key.txt").read()
url = "https://api.rebase.energy/weather/v2/query"
headers = {"Authorization": your_api_key}
params = {
    'model': 'NCEP_GFS',
    'start-date': '2024-10-01',
    'end-date': '2024-10-01',
    'reference-time-freq': '6H',
    'forecast-horizon': 'latest',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'CloudCover'
}
response = requests.get(url, headers=headers, params=params)
response

<Response [401]>

In [7]:
test_date = "2024-01-01"

In [83]:
for col in ["market_index",
                 "day_ahead_price",
                 "imbalance_price",
                 "wind_total_production",
                 "solar_total_production",
                 "solar_and_wind_forecast"
                 ]:
    data = rebase_api_client.get_variable(day = test_date, variable = col)
    print(col)
    print(data.shape)
    data.to_csv("api_files/" + col + "_" + test_date + ".csv")

market_index
(31, 6)
day_ahead_price
(0, 0)
imbalance_price
(29, 4)
wind_total_production
(0, 0)
solar_total_production
(29, 4)
solar_and_wind_forecast
(48, 6)


In [57]:
test_date = "2024-10-01"

In [9]:
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_gfs_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_gfs())

200
200


In [11]:
latest_dwd_Hornsea1.to_netcdf("api_files/dwd_icon_eu_hornsea_" + test_date + ".nc")
latest_gfs_Hornsea1.to_netcdf("api_files/ncep_gfs_hornsea_" + test_date + ".nc")

In [12]:
latest_dwd_pes10 = comp_utils.weather_df_to_xr(rebase_api_client.get_pes10_nwp("DWD_ICON-EU"))
latest_gfs_pes10 = comp_utils.weather_df_to_xr(rebase_api_client.get_pes10_nwp("NCEP_GFS"))

200
200


In [13]:
latest_dwd_pes10.to_netcdf("api_files/dwd_icon_eu_pes10_" + test_date + ".nc")
latest_gfs_pes10.to_netcdf("api_files/ncep_gfs_pes10_" + test_date + ".nc")

In [14]:
latest_dwd_demand = comp_utils.weather_df_to_xr(rebase_api_client.get_demand_nwp("DWD_ICON-EU"))
latest_gfs_demand = comp_utils.weather_df_to_xr(rebase_api_client.get_demand_nwp("NCEP_GFS"))

200
200


In [15]:
latest_dwd_pes10.to_netcdf("api_files/dwd_icon_eu_demand_" + test_date + ".nc")
latest_gfs_pes10.to_netcdf("api_files/ncep_gfs_demand_" + test_date + ".nc")

In [16]:
import pandas as pd

pd.DataFrame(rebase_api_client.get_day_ahead_demand_forecast()).to_csv("api_files/demand_forecast_" + test_date + ".csv")

<Response [200]>


In [17]:
import pandas as pd

pd.DataFrame(rebase_api_client.get_margin_forecast()).to_csv("api_files/margin_forecast_" + test_date + ".csv")

<Response [200]>
